In [1]:
import serial, time, sys

In [82]:
# Configure xWR by serial port
serial_port_CLI = '/dev/ttyACM0'
file_cfg = '/home/dingrong/Code/ackermann_car_nav/src/mmwave_radar/config/xwr18xx.cfg'
CLIport = serial.Serial(serial_port_CLI, 115200, timeout=1)
if CLIport.is_open:
    config = [line.rstrip('\r\n') for line in open(file_cfg)]
    for i in config:
        if(i == '' or i[0] == '%'):
            continue
        if(i == 'sensorStart'):
            break
        CLIport.write((i+'\n').encode())
        print('>>> ' + i)
        time.sleep(0.01)
else:
    print("Open serial port failed.")

>>> sensorStop
>>> flushCfg
>>> dfeDataOutputMode 1
>>> channelCfg 15 7 0
>>> adcCfg 2 1
>>> adcbufCfg -1 0 1 1 1
>>> profileCfg 0 77 267 7 57.14 0 0 70 1 256 5209 0 0 30
>>> chirpCfg 0 0 0 0 0 0 0 1
>>> chirpCfg 1 1 0 0 0 0 0 4
>>> chirpCfg 2 2 0 0 0 0 0 2
>>> frameCfg 0 2 16 10 100 1 0
>>> lowPower 0 0
>>> guiMonitor -1 1 0 0 0 0 0
>>> cfarCfg -1 0 2 8 4 3 0 15 1
>>> cfarCfg -1 1 0 4 2 3 1 15 1
>>> multiObjBeamForming -1 1 0.5
>>> clutterRemoval -1 0
>>> calibDcRangeSig -1 0 -5 8 256
>>> extendedMaxVelocity -1 0
>>> lvdsStreamCfg -1 0 1 0
>>> compRangeBiasAndRxChanPhase 0.0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
>>> measureRangeBiasAndRxChanPhase 0 1.5 0.2
>>> CQRxSatMonitor 0 3 5 121 0
>>> CQSigImgMonitor 0 127 4
>>> analogMonitor 0 0
>>> aoaFovCfg -1 -90 90 -90 90
>>> cfarFovCfg -1 0 0 8.92
>>> cfarFovCfg -1 1 -1 1.00
>>> calibData 0 0 0


In [83]:
CLIport.write(('sensorStart\n').encode())
print('>>> sensorStart')

>>> sensorStart


In [80]:
CLIport.write(('sensorStop\n').encode())
print('>>> sensorStop')

>>> sensorStop


In [85]:
196608/1470

133.74693877551022

In [21]:
import numpy as np

In [37]:
adc1 = np.fromfile("/home/dingrong/Code/DCA1000/DCA1000-source-code/src/mmwave_radar/data/adc_1.bin", np.int16)
adc2 = np.fromfile("/home/dingrong/Code/DCA1000/DCA1000-source-code/src/mmwave_radar/data/adc_2.bin", np.int16)

In [38]:
adc1.shape, adc2.shape

((98304,), (98304,))

In [8]:
# 每帧16个chirp，每个chirp 256个sample，每个sample有2个整数，每个整数有2个字节，共有3个tx发射信号，4个rx
print("bytes per frame:", 16*256*2*2*3*4)
print("bytes per chirp:", 256*2*2)


bytes per frame: 196608
bytes per chirp: 1024


In [42]:
# bag文件解析，并理解ros中旋转是
import rosbag
import numpy as np
from sensor_msgs import point_cloud2

In [44]:
bag = rosbag.Bag("/home/dingrong/Code/ackermann_car_nav/person_2023-05-11-13-11-47.bag", 'r')
bag_data = bag.read_messages()

livox_odom = {
    'time': [],
    'position_x': [],
    'position_y': [],
    'position_z': [],
    'orientation_x': [],
    'orientation_y': [],
    'orientation_z': [],
    'orientation_w': []
}
mmwave_pc = {
    'time': [],
    'x': [],
    'y': [],
    'z': [],
    'vel': []
}


In [19]:
for topic, msg, t in bag_data:
    if topic == '/aft_mapped_to_init_high_frec':
        livox_odom['time'].append(t.to_sec())
        livox_odom['position_x'].append(msg.pose.pose.position.x)
        livox_odom['position_y'].append(msg.pose.pose.position.y)
        livox_odom['position_z'].append(msg.pose.pose.position.z)
        livox_odom['orientation_x'].append(msg.pose.pose.orientation.x)
        livox_odom['orientation_y'].append(msg.pose.pose.orientation.y)
        livox_odom['orientation_z'].append(msg.pose.pose.orientation.z)
        livox_odom['orientation_w'].append(msg.pose.pose.orientation.w)
    # if topic == '/livox/imu':
    #     print(msg)
    # if topic == '/mmwave_radar_point_cloud':
    #     print(msg)

In [52]:
def transform(radar_xy, delta_x, delta_y, yaw):
    """Transform xy from radar coordinate to the world coordinate.
    Inputs:
        radar_xy: Nx2，雷达坐标系下的点云
        delta_x, delta_y: 雷达在世界坐标系中的坐标
        yaw: 雷达坐标系逆时针旋转yaw度与世界坐标系重合
    Return:
        world_xy: Nx2
    """
    yaw = yaw * np.pi / 180
    rotation_matrix = np.array([[np.cos(yaw), -np.sin(yaw)], [np.sin(yaw), np.cos(yaw)]])
    translation_vector = np.array([delta_x, delta_y]).reshape(1, -1)
    world_xy = radar_xy.dot(rotation_matrix) + translation_vector
    return world_xy

In [46]:
def nlos_proc():
    for topic, msg, t in bag_data:
        if topic == '/mmwave_radar_point_cloud':
            points = point_cloud2.read_points_list(
                msg, field_names=['x', 'y', 'z', 'vel']
            )
            mmwave_pc['time'].append(t.to_sec())
            x_pos = [p.x for p in points]
            y_pos = [p.y for p in points]
            z_pos = [p.z for p in points]
            vel = [p.vel for p in points]
            point_cloud = np.array([x_pos, y_pos, z_pos, vel]).T
            point_cloud[:, :2] = transform(point_cloud[:, :2], -0.043, 0.13, 60)
            yield point_cloud

[Point(x=0.0, y=0.0, z=-0.0, vel=0.0), Point(x=-0.04234445467591286, y=0.009775987826287746, z=-0.006883116438984871, vel=0.0), Point(x=-0.02767377346754074, y=0.0803704708814621, z=0.022776076570153236, vel=0.0), Point(x=-0.0656684935092926, y=0.1092907264828682, z=0.03416411578655243, vel=0.0), Point(x=-0.1339641511440277, y=0.10466426610946655, z=-0.04555215314030647, vel=0.0), Point(x=-0.5653719305992126, y=1.0199580192565918, z=-0.22668108344078064, vel=0.0), Point(x=-0.5696019530296326, y=1.071265459060669, z=-0.21393455564975739, vel=0.0), Point(x=-0.5721598267555237, y=1.122926950454712, z=-0.1996103823184967, vel=0.0), Point(x=-0.5270281434059143, y=1.1837246417999268, z=-0.2518678605556488, vel=0.0), Point(x=-0.3663332760334015, y=1.2775559425354004, z=-0.3068332374095917, vel=0.0), Point(x=-0.35507723689079285, y=1.3251662254333496, z=-0.3167310953140259, vel=0.0), Point(x=-0.3661733865737915, y=1.3665777444839478, z=-0.3266289234161377, vel=0.0), Point(x=1.8680624961853027,

In [51]:
msg.

0.0

In [40]:
def kalman(z_measure,x_last=0,p_last=0,Q=0.5,R=0.5):
    x_mid = x_last
    p_mid = p_last + Q
    kg = p_mid/(p_mid + R)
    x_now = x_mid + kg*(z_measure - x_mid)
    p_now = (1-kg)*p_mid
    p_last = p_now
    x_last = x_now
    return x_now,p_last,x_last

x_last = 0
p_last = 0
Q = 0.01  #系统噪声
Rk = 0.5  #测量噪声

vz_tmp = []
vzs = livox_odom['position_x']
for i in range(len(vzs)):
    pred,p_last,x_last = kalman(vzs[i],x_last,p_last,Q,Rk)
    vz_tmp.append(pred)

In [41]:
vz_tmp

[0.0006190754756210386,
 0.0018454164971380676,
 0.003639441711488312,
 0.0058598321453215375,
 0.008324259756607794,
 0.011017888100699591,
 0.013844854547424679,
 0.0166615856362297,
 0.019407455208307608,
 0.021979080252755398,
 0.024438884406490183,
 0.026769829426044334,
 0.028948825271693207,
 0.031115092095493412,
 0.03331558796186754,
 0.0353957651477763,
 0.037247667322962384,
 0.03897530350545866,
 0.04052507714190103,
 0.042119078842446006,
 0.04356773585821386,
 0.04513436449878805,
 0.04673181432847727,
 0.04825724871822125,
 0.04978337963871959,
 0.05137132586680477,
 0.05293467897475123,
 0.054506825705159835,
 0.05594455030633712,
 0.057354624553004906,
 0.058618997807172946,
 0.05971537534247495,
 0.06080641915153809,
 0.061950840495710376,
 0.06303567313044971,
 0.06411561626621404,
 0.06522733351503528,
 0.06646237030872282,
 0.06770307147514382,
 0.0690154505994615,
 0.07010782896193613,
 0.0713301883841906,
 0.07327330076948542,
 0.07466016285429057,
 0.07580518296

In [35]:
livox_odom['position_x']

[0.03157284925667297,
 0.03280749179286503,
 0.03451829408937753,
 0.035501121495193674,
 0.035801913426276144,
 0.037346895635459676,
 0.038904105152299515,
 0.039868424737315535,
 0.040819799739286934,
 0.041218850172454254,
 0.042276112800047255,
 0.04327906878698288,
 0.04411001190165542,
 0.04598700179676692,
 0.04827050741453632,
 0.04942558312956193,
 0.04966625690518631,
 0.0505104550335972,
 0.050838892136615126,
 0.052701115737926074,
 0.05316702185373422,
 0.0555008351282167,
 0.05729107117181423,
 0.05833245245591429,
 0.05985713492672202,
 0.06184837235988482,
 0.06324594548413166,
 0.06487342539579898,
 0.06542294455551273,
 0.06664937178374077,
 0.06695241384827987,
 0.06694094063874219,
 0.06799638300040765,
 0.06949220572929043,
 0.07018411384381582,
 0.07123164711496065,
 0.07255258433957378,
 0.07460006581050105,
 0.07587799581411334,
 0.07766258292480321,
 0.07730535171178286,
 0.07938409788150688,
 0.0860760762655161,
 0.08379789179600353,
 0.08334945248329502,
 0.

In [7]:
# 参数解析
xwr_cfg = "/home/dingrong/Code/ackermann_car_nav/src/mmwave_radar/config/xwr18xx.cfg"
callback_args = {}
for line in open(xwr_cfg):
    line = line.rstrip('\r\n')
    if line.startswith('profileCfg'):
        config = line.split(' ')
        callback_args['idleTime'] = config[3]
        callback_args['adcStartTime'] = config[4]
        callback_args['rampEndTime'] = config[5]
        callback_args['freqSlop'] = config[8]
        callback_args['txStartTime'] = config[9]
        callback_args['numAdcSamples'] = config[10]
        callback_args['digOutSampleRate'] = config[11]
    elif line.startswith('frameCfg'):
        config = line.split(' ')
        callback_args['numLoops'] = config[3]
        callback_args['numFrames'] = config[4]
        callback_args['framePeriodicity'] = config[5]
        callback_args['frameTriggerDelay'] = config[7]

callback_args['numTx'] = 3
callback_args['numRx'] = 4
callback_args['virt_ant_azimuth'] = 8
callback_args['virt_ant_elevation'] = 2
callback_args['angle_range_azimuth'] = 90
callback_args['angle_range_elevation'] = 15
callback_args['angle_res'] = 1


In [8]:
callback_args

{'idleTime': '267',
 'adcStartTime': '7',
 'rampEndTime': '57.14',
 'freqSlop': '70',
 'txStartTime': '1',
 'numAdcSamples': '256',
 'digOutSampleRate': '5209',
 'numLoops': '16',
 'numFrames': '0',
 'framePeriodicity': '100',
 'frameTriggerDelay': '0'}

In [9]:
"""
mmwave_proc是为了实时生成点云用的，
主要矛盾在于，我现在的一些参数是写死的，如果修改了配置文件，现在的代码有些东西就要修改
至于cfar的参数，我完全可以把原始数据保存下来，离线处理
"""

5186.24